In [18]:
# Import necessary libraries
import torch
import sys
import os

# Add the path to your sparse_lo module
sys.path.append('/Users/matthew/Documents/Efficient Gaussian Process on Graphs/Efficient_Gaussian_Process_On_Graphs/efficient_graph_gp_sparse/utils_sparse')

# Import your SparseLinearOperator
from sparse_lo import SparseLinearOperator

In [19]:
# Create a simple test sparse matrix
print("Creating a simple sparse CSR tensor...")

# Create a 4x4 sparse matrix with some non-zero elements
# Matrix structure:
# [1, 0, 2, 0]
# [0, 3, 0, 0] 
# [0, 0, 0, 4]
# [5, 0, 0, 6]

indices = torch.tensor([[0, 0, 1, 2, 3, 3], [0, 2, 1, 3, 0, 3]])  # row, col indices
values = torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0, 6.0])
size = (4, 4)

sparse_coo = torch.sparse_coo_tensor(indices, values, size)
sparse_csr = sparse_coo.to_sparse_csr()

print(f"Sparse CSR tensor:\n{sparse_csr}")
print(f"\nDense version:\n{sparse_csr.to_dense()}")

Creating a simple sparse CSR tensor...
Sparse CSR tensor:
tensor(crow_indices=tensor([0, 2, 3, 4, 6]),
       col_indices=tensor([0, 2, 1, 3, 0, 3]),
       values=tensor([1., 2., 3., 4., 5., 6.]), size=(4, 4), nnz=6,
       layout=torch.sparse_csr)

Dense version:
tensor([[1., 0., 2., 0.],
        [0., 3., 0., 0.],
        [0., 0., 0., 4.],
        [5., 0., 0., 6.]])


In [20]:
# Create SparseLinearOperator and test basic properties
print("Creating SparseLinearOperator...")

sparse_lo = SparseLinearOperator(sparse_csr)

print(f"Operator size: {sparse_lo.size()}")
print(f"Operator shape: {sparse_lo.shape}")
print(f"Number of non-zero elements: {sparse_csr._nnz()}")

Creating SparseLinearOperator...
Operator size: torch.Size([4, 4])
Operator shape: torch.Size([4, 4])
Number of non-zero elements: 6


In [21]:
slo1 = SparseLinearOperator(sparse_csr)
slo2 = SparseLinearOperator(sparse_csr)
# Test addition
print("Testing addition of SparseLinearOperators...")
slo_sum = slo1 + slo2
print(f"Sum operator size: {slo_sum.size()}")
print(f"Sum operator shape: {slo_sum.shape}")

# test multiplication
print("Testing multiplication of SparseLinearOperators...")
slo_prod = slo1 @ slo2
print(f"Product operator size: {slo_prod.size()}")
print(f"Product operator shape: {slo_prod.shape}")

# Create vector with gradient tracking enabled
vec = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
print(f"Operating on vector {vec}...")
result = slo_prod @ vec
print(f"Result of operation: {result}") 

# Test autograd
print("Testing autograd with SparseLinearOperator...")
# Clear any existing gradients
if vec.grad is not None:
    vec.grad.zero_()

result = slo_prod @ vec
result.sum().backward()
print(f"Gradient of the vector: {vec.grad}")

# Verify the autograd functionality by calculating the gradient manually
# For (A @ B) @ vec, gradient w.r.t vec is (A @ B)^T @ ones_vector
# Since we're taking sum().backward(), the upstream gradient is a vector of ones
with torch.no_grad():
    # Get the dense matrices for manual calculation
    A_dense = slo1.sparse_csr_tensor.to_dense()
    B_dense = slo2.sparse_csr_tensor.to_dense()
    AB_dense = A_dense @ B_dense
    ones_vector = torch.ones_like(result)
    expected_grad = AB_dense.t() @ ones_vector

print(f"Expected gradient: {expected_grad}")
print(f"Gradient matches expected: {torch.allclose(vec.grad, expected_grad, atol=1e-6)}")

Testing addition of SparseLinearOperators...
Sum operator size: torch.Size([4, 4])
Sum operator shape: torch.Size([4, 4])
Testing multiplication of SparseLinearOperators...
Product operator size: torch.Size([4, 4])
Product operator shape: torch.Size([4, 4])
Operating on vector tensor([1., 2., 3., 4.], requires_grad=True)...
Result of operation: tensor([ 39.,  18., 116., 209.], grad_fn=<MatmulBackward>)
Testing autograd with SparseLinearOperator...
Gradient of the vector: tensor([56.,  9., 12., 68.])
Expected gradient: tensor([56.,  9., 12., 68.])
Gradient matches expected: True
